In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 ## [範例重點]
 了解 sklearn 中，GridSearchCV 的使用方法與原理

In [2]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier


 ### 作業
 請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [3]:
breast_cancer = datasets.load_breast_cancer()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(
    breast_cancer.data, breast_cancer.target, test_size=0.25, random_state=4)


In [4]:
# 先看看使用預設參數得到的結果
clf1 = RandomForestClassifier(n_estimators=30, max_depth=6, bootstrap=True)
clf1.fit(x_train, y_train)
y_pred = clf1.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)


Accuracy:  0.9300699300699301


In [5]:
# 設定要訓練的超參數組合
n_estimators = [20, 30, 40, 50]
max_depth = [3, 4, 5, 6]
bootstrap=[True, False]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, bootstrap=bootstrap)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf1, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)


Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    1.5s finished


In [6]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best Accuracy: -0.032864 using {'bootstrap': False, 'max_depth': 6, 'n_estimators': 40}


In [7]:
# 使用最佳參數重新建立模型
clf_hat = RandomForestClassifier(n_estimators=grid_result.best_params_['n_estimators'], 
                                    max_depth=grid_result.best_params_['max_depth'], 
                                    bootstrap=grid_result.best_params_['bootstrap'])
clf_hat.fit(x_train, y_train)
y_pred = clf_hat.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)



Accuracy:  0.9230769230769231
